# 전처리 


In [2]:
import numpy as np
import pandas as pd

## 데이터 인코딩(Encoding)

### 데이터 타입 

데이터는 크게 2가지 형태로 나뉩니다.
- 수치형(Numerical)
- 범주형(Categorical)

> 📈 **Numerical (수치형) 데이터**
> - Discrete (이산형, 불연속형) 데이터  
     : 숫자로 셀 수 있는 것, 특정 값, 정수형태 (1개, 2명 등)
>- Continuous (연속형) 데이터  
     : 측정해야 하는 것, 연속 범위 내 실수값 (온도(21.3℃), 무게(72.5kg), 길이(183m), 비율(57%) 등)

> 📊 **Categorical (범주형) 데이터**
> - Nominal (범주형) 데이터  
     : 순서가 없는 것 (성별, 동물의 종류 등)  
> - Ordinal (순서형) 데이터  
     : 순서가 있는 것 (수능 등급, 좌석 등급, 순위 등)

하지만, 기본적인 머신러닝 모델은 categorical_data 를 입력값으로 활용할 수 없습니다.
따라서, categorical data 를 numerical 하게 변환해주는 전처리 작업(preprocessing)이 선행되어야합니다. 이러한 전처리작업을 인코딩 작업이라합니다. 

### 인코딩 방법 

- `sklearn` LabelEncoder 
- `pandas` pd.get_dummies
- `pandas` `map` 이용하기  

- pd.get_dummies
#### ③ **pandas**의 **get_dummies**  이용하기 🔩

위와 같은 인코딩(전처리) 방법 두 개는 간단하고 직관적이지만, 한 가지 문제가 있습니다 ⛔

> **[ 매우 싫다 -> 1, 보통이다 -> 3 , 매우 좋다 -> 5점 ]** 처럼  
ordianal한 데이터에는 적합하지만, (*순서 또는 등급 有)

> **[ 빨강 -> 1, 파랑 -> 2, 초록 -> 3 ]** 처럼  
nominal한 데이터에는 적합하지 않습니다. (*순서 또는 등급 無)

> 빨강, 파랑, 초록이라는 색깔들 사이에는 어떠한 순서나 등급도 없지만,  
위의 방법들로 변환하면 초록값이 가장 크다던지,  
"(초록값) - (파랑값) = (빨강값)" 등의 **관계**가 생기기 때문입니다.  
머신러닝 모델은 변환된 숫자를 바탕으로 학습하기 때문에, 잘못하면 위의 관계(특성)를 학습해 버립니다 😭😭  
이는 논리적으로 문제가 있을 뿐만 아니라, 실제 모델의 성능도 잘 안나오게 됩니다. 

**💡 따라서 nominal한 데이터 변환을 위해서는**, 숫자이기는 하지만 **서로 '관계없는' 숫자로 변환**해 주어야합니다.  

> - `pd.get_dummies(데이터셋.타겟열, prefix='타겟열')`

![예시](https://www.mwbrady.com/post/encodingcategoricaldata/Untitled-5f6d339d-ce4a-4533-8b67-c4e688b85ce0.png)

- map 

### LabelEncoder

In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.DataFrame({
    'pName':['냉장고','TV','에어컨','냉장고','노트북','공기청정기'],
    'price':[100,80,150,100,50,40]
})

df

,pName,price
0,냉장고,100
1,TV,80
2,에어컨,150
3,냉장고,100
4,노트북,50
5,공기청정기,40


In [9]:
# 전처리 
le = LabelEncoder()
decode_name = le.fit_transform(df.pName)# 각 값들과 코드의 매핑 관계 생성 후 변환 

In [11]:
df['pid'] = decode_name
df

,pName,price,pid
0,냉장고,100,2
1,TV,80,0
2,에어컨,150,4
3,냉장고,100,2
4,노트북,50,3
5,공기청정기,40,1


In [13]:
print('Encoding Class : ',le.classes_)
print('Decoding Class: ',decode_name )

Encoding Class :  ['TV' '공기청정기' '냉장고' '노트북' '에어컨']
Decoding Class:  [2 0 4 2 3 1]


### OneHotEncoder

In [28]:
df_dummies = pd.get_dummies(df, prefix='pName')  # origin에 속한 카테고리(원산지)별로 새로운 comlumn(열)이 생김
df_dummies

,price,pid,map_name,pName_TV,pName_공기청정기,pName_냉장고,pName_노트북,pName_에어컨
0,100,2,2,0,0,1,0,0
1,80,0,0,1,0,0,0,0
2,150,4,4,0,0,0,0,1
3,100,2,2,0,0,1,0,0
4,50,3,3,0,0,0,1,0
5,40,1,1,0,1,0,0,0


In [27]:
# pd.concat
pd.concat([df,df_dummies], axis=1)

,pName,price,pid,map_name,price,pid,pName_TV,pName_공기청정기,pName_냉장고,pName_노트북,pName_에어컨
0,냉장고,100,2,2,100,2,0,0,1,0,0
1,TV,80,0,0,80,0,1,0,0,0,0
2,에어컨,150,4,4,150,4,0,0,0,0,1
3,냉장고,100,2,2,100,2,0,0,1,0,0
4,노트북,50,3,3,50,3,0,0,0,1,0
5,공기청정기,40,1,1,40,1,0,1,0,0,0


### Map

In [17]:
df['map_name'] = df.pName.map({'TV':0,'공기청정기':1,'냉장고':2,'노트북':3,'에어컨':4 })

In [18]:
df

,pName,price,pid,map_name
0,냉장고,100,2,2
1,TV,80,0,0
2,에어컨,150,4,4
3,냉장고,100,2,2
4,노트북,50,3,3
5,공기청정기,40,1,1


<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i> 실제데이터에 인코딩해보기 
  </h2>
</div>

- Excercise 0: smoker,day, time 컬럼 드랍하여 tips 만들기 
- Excercise 1: total_bil의 범주형 데이터('sex') 를 수치형으로 바꾸기 
- Excercise 2: total_bil을 y로 // 나머지를 x 로 나누기 
- Excercise 3: x를 minmax scale 하기 
- Excercise 4: train_test_split 비율 0.3
- Excercise 5: LinearRegression 모델로 `total_bill` 예측해보기 
- Excercise 6: r2 스코어로 평가하기

In [30]:
import seaborn as sns

tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
